# Comparison of regression coefficients

In our simplified setting, we have observations of the type $(x,y,c)$ where $(x,y)$
are both continuous real-valued variables and $c$ a categorical variable with two categories, 0 and 1. We have separately considered linear models of the form

$$
y = \beta_0 + \beta_1 x
$$

on the subsets of (x, y) where c = 0 and c = 1, respectively. When we wish to refer to the two different models, we will use the notation $\beta_0^i, \beta_1^i$ where $i \in \{0, 1 \}$ depending on the value of $c$. Our question can then be phrased: given the data, are the differences in $\beta_0^i$ and $\beta_1^i$ across $i$ statistically significant?

## Likelihood-ratio test

Consider the following linear model for y across all data:

$$
y = \beta_0 + \beta_1 x + \beta_2 c + \beta_3 x c
$$

Note that a single model of this kind is equivalent to both of our simpler linear models from before, where the simpler ones can be recovered by taking $c = 0$ or $c = 1$. What this model makes clear is that the deviation of $\beta_2$ and $\beta_3$ from 0 somehow accounts for the difference in our linear models.

Let the null hypothesis $H_0$ be $\beta_2, \beta_3 = 0$, or that there is no difference between our two linear models. The statistic we are interested in is

$$
\Lambda(d) := \frac{\sup_{\beta_0, \beta_2} \{ L(d | \beta_0, \beta_1, \beta_2 = \beta_3 = 0) \}}{ \sup_{\beta_0, \beta_1, \beta_2, \beta_3} \{L(d | \beta_0, \beta_1, \beta_2, \beta_3) \} }
$$

where $L(d | \beta)$ represents the likelihood of observing the data $d$ given that the model relating $y, x$ and $c$ is true for a specific value of $\beta$. Intuitively, $\Lambda(d)$ is small when allowing for the change in $c$ (by freeing $\beta_2, \beta_3$ from being uniquely 0) allows for a much more 'likely' model.

Suppose we knew for all threshold values $t$ of the statistic $\Lambda$ the probability $P(\Lambda < t | H_0)$. Then, given a significance level $\alpha$, we could find $t_\alpha$ such that $P(\Lambda < t_\alpha | H_0) = \alpha$ and define a test:

* If $\Lambda < t_\alpha$, reject $H_0$.
* If $\Lambda \geq t_\alpha$, fail to reject $H_0$. 

It is at this point that we invoke a crucial theorem:

Theorem (Wilks): In the limit as $n = \left| d \right|$ tends to infinity, $-2 \log \Lambda(d)$ under the null hypothesis $H_0$ is distributed as a $\chi^2$ distribution with degrees of freedom equal to ($df_{gen} - df_{sp}$), where $df_{gen}$ is the degrees of freedom of the general model (all parameters free) and $df_{sp}$ is the degrees of freedom of the null-hypothesis model (some parameters fixed).

In our case, where the general model has $4$ degrees of freedom and the specific model has $2$, this means that $-2 \log \Lambda(d)$ should be approximately distributed like a $\chi^2$ distribution on $4-2 = 2$ degrees of freedom.

## Experiments/Implementation 

Thanks to some built-in packages, these likelihood tests are very easy to code. Let us first create some fake data, where the categorical variable $c$ has no influence on $y$

In [17]:
import numpy as np
import pandas as pd
from statsmodels.formula.api import ols

np.random.seed(77)
x_data = np.random.randn(100)
c_data = np.random.choice([0,1], 100)
error = 0.1*np.random.randn(100)
y_data = x_data + error

data = pd.DataFrame({'x': x_data, 'c': c_data, 
                     'xc': np.multiply(x_data, c_data), 'y': y_data})

'''
The 'ols' function takes a proposed linear relationship between columns (it
will by default include an intercept term as well) of the form 
'y ~ x_1 + x_2 + ... + x_n' and returns a model object, on which the 'fit' 
function runs to produce an OLSResults object.
'''
no_c_model = ols("y ~ x", data=data).fit()
with_c_model = ols("y ~ x + c + xc", data=data).fit()

'''
The 'compare_lr_test(submodel)' on a OLSResults object takes a submodel representing
a null hypothesis, and returns the tuple (-2 log Lambda, p-value, df_diff)
where df_diff is the difference in degrees of freedom between the model and its
submodel.
'''
(stat, p_value, df_diff) = with_c_model.compare_lr_test(no_c_model)
print('The p_value of this test was: {}'.format(p_value))


The p_value of this test was: 0.5662137730439274


As may be expected, the data was generated to be independent of $c$. Thus we
find that the p_value is large (there is no evidence to reject the null
hypothesis) and we cannot reject the assumption that the category has no influence on the regressions.

For contrast, now we generate some random data, where the categorical value of $c$ DOES have an influence on the linear relationship between $x$ and $y$.

In [19]:
np.random.seed(777)
x_data = np.random.randn(100)
c_data = np.random.choice([0,1], 100)
error = 0.1*np.random.randn(100)
y_data = x_data + c_data + error

data = pd.DataFrame({'x': x_data, 'c': c_data, 
                     'xc': np.multiply(x_data, c_data), 'y': y_data})

no_c_model = ols("y ~ x", data=data).fit()
with_c_model = ols("y ~ x + c + xc", data=data).fit()

(stat, p_value, df_diff) = with_c_model.compare_lr_test(no_c_model)
print('The p_value of this test was: {}'.format(p_value))

The p_value of this test was: 1.80642536917689e-76


In this case, the log-likelihood test determines with obscene confidence (look at that p-value!) that the two models are different; at any reasonable level of $\alpha$, we choose to reject the null hypothesis that there is no difference across values of the category $c$. 

Consider a likelihood-ratio test with a significance level of $\alpha = 0.05$. Our first experiment will be simulating data of the form

$$
y = x + \beta_3 x c + \epsilon(x)
$$

where $\epsilon$ is some normally distributed error with mean 0 and standard deviation $\sigma^2$. If we first take $\beta_3 = 0$, then by Wilks' theorem, we should expect to see rejection only $5$ percent of the time.

In [13]:
SAMPLE_SIZE = 100
NUM_SAMPLES = 1000

reject_count = 0
np.random.seed(7777)
for i in range(NUM_SAMPLES):
    x_data = np.random.randn(SAMPLE_SIZE)
    c_data = np.random.choice([0, 1], SAMPLE_SIZE)
    error = 0.1*np.random.randn(SAMPLE_SIZE)
    
    y_data = x_data + error
    
    data = pd.DataFrame({'x': x_data, 'y': y_data, 
                         'c': c_data, 'xc': np.multiply(x_data, c_data)})
    
    no_c_model = ols("y ~ x", data=data).fit()
    with_c_model = ols("y ~ x + c + xc", data=data).fit()
    p_value = with_c_model.compare_lr_test(no_c_model)[1]
    if p_value < 0.05:
        reject_count = reject_count + 1

reject_count/1000

0.049